# Bayes error rate
In this notebook, we provide an explanation for the low accuracy of our binary classification models. In a nutshell, the problem lies within our (final, cleaned) dataset-- there is an *intrinsic* upper bound on the accuracy we could have hoped to achieve, and this upper bound is not much higher than the accuracy we were actually able to obtain (so, in other words, our models actually performed fairly well relative to the theoretical best accuracy). The following toy example already serves to illustrate the key idea.

### A toy example
Suppose we have a binary classification problem, wherein we have a dataset with some features $X$ and a binary target variable $Y$. Suppose the dataset contains exactly two observations, both of which have identical features $X$, but $Y=1$ for the first observation and $Y=0$ for the second observation. Then, *any classifier* we could train would make the same predictions for the two observations, becuase they are completely indistinguishable by the features used to train the classifier. Thus, whether or not this classifier predicts $1$ or $0$, we are guaranteed to have an error rate of $50\%$!

### Generalizing the example
This guaranteed error rate generalizes naturally to binary classification problems involving larger datasets (and discrete features). Namely:
- Suppose we have a dataset with features $X$ (treating multiple scalar-valued features as a single vector-valued feature) and binary target $Y$. 
- Suppose there is some value of $X$ (i.e. a row vector) which arises as the set of features for a group of $m$ observations. 
- Within this group, suppose $m_1$ observations belong to class $Y=1$, and $m_0$ belong to $Y=0$ (so $m = m_1 + m_0$). 

Then, by the same logic as above, *any* binary classifier trained on the dataset would make the same, constant prediction for all $m$ members in the group. In fact, if the classifier is sensibly trained, it would predict the majority class within the group, leading to misclassifications for all memebers in the minority class within the group. Thus, in the best case scenario, out of $m$ observations, $\min(m_0,m_1)$ would be misclassifications, so this group would contribute a (theoretically unavoidable) error rate of
\begin{equation*} \dfrac{ \min(m_0,m_1) }{\# \{\textup{all observations} \}}. \end{equation*}
This idea (made rigorous) leads to the *Bayes Error Rate* for a binary classification problem (by which mean the problem of predicting $Y$ from $X$ for a *given* dataset)-- it is the least possible error that can be achieved by *any* classifier for the given problem.

### Bayes Classifier
The Bayes classifier $C^{\textup{Bayes}}$ is the most natural binary classifier that can be conceived of for a given binary classification problem. In a nutshell, given an observation $X=x$, the classifier simply groups together all observations with the same value of $X$ (i.e. identical feature rows in the dataset) and predicts $Y$ according to the majority class within the group. Note: grouping by $X=x$ and computing the probability of each class amounts to computing the conditional probabilities $Pr(Y=1|X=x)$ and $Pr(Y=0|X=x)$. Thus, for each observation in the group corresponding to $X=x$, the Bayes Classifier predicts as follows:
\begin{equation*} C^{\textup{Bayes}}(x) \coloneqq \max \big(Pr(Y=1|X=x), Pr(Y=0|X=x) \big). \end{equation*}
Why do we bring up the Bayes Classifier? As might already be apparent from its design and naturality, the Bayes Classifier is the "best possible" binary classifier in the sense that the error rate equals the theoretical minimum, i.e. the Bayes Error Rate. 

### Computing the Bayes Classifier
One expects or assumes that the dataset contains a (more or less) representative sample of the distribution of $X$, so  one is typically not interested in predicting $Y$ based on any specific value of $X$, but rather, on the whole range of possible values of $X$. 

Thus, in practice, directly computing the Bayes Classifier as defined may not be very useful, and instead, one is interested in *estimating* the Bayes Classifier, which amounts to estimating the prior distribution of $Y$ given that of $X$. 

Nevertheless, directly computing the Bayes Classifier will at least yield a *strict lower bound* on the "true" Bayes Error Rate (i.e. the error rate for the "true" distribution of $X$ and $Y$). Indeed, there will likely be lots of "singleton" observations which have a unique vector of feature values (especially if $X$ has continuous features). These singletons don't contribute to the error of the Bayes Classifier because there is no minority class to be misclassified within the singleton. By contrast, any practicable classifier trained on the distribution of $X$ will likely accumulate error on these observations (unless it is dangerously overfit!) 

### Bayes Classifier for our dataset
That said, our dataset is quite small (around 9000 observations), so even though our feature-vectors could potentially have millions of distinct values (specifically, $20$ to the power of the number of courses), there are at most 9000 of them! So, let's go ahead and compute the Bayes Classifier (by hand) for our dataset as follows:
1. First, we group together out dataframe by the unique values of the vector of courses (this is out $X$).
2. For each group, we add columns:
    - `COUNT(X)`: the size of the group
    - `Pr(X)`: the probability of the observation occuring, i.e. `COUNT(X)` divided by total size of the dataset.
    - `Pr(Y=1|X)`: Probability of the `Y=1` class within the group.
    - `ERROR(Y|X)`: Unavoidable (conditional) error introduced when predicting on the group, equal to $\min \big( Pr(Y=1|X), Pr(Y=0|X) \big)$.
    - `ERROR(Y|X) * Pr(X)`: Unavoidable (absolute) error introduced to the whole dataset because of the group.
    - `CUMULATIVE_ERROR`: A cumulative sum of the above errors as we go down the dataset (after sorting in descending order of the absolute error).
    - The final value of `CUMULATIVE_ERROR` is nothing but the expected value of the conditional errors (taken over the distribution of $X$).
3. Finally, we split the dataframe into two parts, the first having groups of size greater than one (so there is at all some potential for misclassification), and the second consisting of singleton groups (which don't contribute to error of the Bayes Classifier, but are likely to induce errors on other binary classifiers trained on the dataset).

In [17]:
import pandas as pd
import numpy as np
import json
import sys
from pathlib import Path

# Add root directory to Python path
root_dir = Path.cwd().parent  # Go up one level from Notebooks folder
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

# Now import from Source Code directory
sys.path.append(str(root_dir / 'Source Code'))
from modeller import *

#import the dataset
df = pd.read_csv('../Data/Datasets/dataset_engineered.csv')

#automatically reload all modules
%load_ext autoreload
%autoreload 2

#load course dict
with open('../Data/Dictionaries/crse_dict.json') as f:
    crse_dict = json.load(f)
courses = list(crse_dict.keys())

In [42]:
grouped_df, singletons_df = get_bayes_error(df,courses,save=True)

We visualize the top 10 rows of the grouped dataset to see the groups contributing the largest error to our problem.

In [47]:
grouped_df.head(10)

,165,166,265,143,140,104,150,207,201,317,...,495,314,500,304,COUNT(X),Pr(X),Pr(Y=1|X),ERROR(Y|X),ERROR(Y|X) * Pr(X),CUMULATIVE_ERROR
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,324,0.035290,0.472222,0.472222,0.016665,0.016665
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,344,0.037469,0.412791,0.412791,0.015467,0.032132
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,214,0.023309,0.602804,0.397196,0.009258,0.041390
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,189,0.020586,0.412698,0.412698,0.008496,0.049886
4,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,177,0.019279,0.604520,0.395480,0.007624,0.057510
5,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,146,0.015902,0.390411,0.390411,0.006208,0.063719
6,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,159,0.017318,0.333333,0.333333,0.005773,0.069491
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,146,0.015902,0.643836,0.356164,0.005664,0.075155
8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,136,0.014813,0.683824,0.316176,0.004684,0.079839
9,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,95,0.010347,0.442105,0.442105,0.004575,0.084413


In [ ]:
print(f'There are {len(grouped_df)} groups of feature values of size greater than 1.')
print(f'There are {len(singletons_df)} singleton groups of feature values.')
print(f'Total number of groups: {len(grouped_df)+len(singletons_df)}')
print(f'Lower bound on Bayes Error Rate: {grouped_df.iloc[-1]["CUMULATIVE_ERROR"]}')
grouped_prob = grouped_df['Pr(X)'].sum()
print(f'Error contributed by {grouped_prob} of the whole dataset.')

There are 578 groups of feature values of size greater than 1.
There are 2002 singleton groups of feature values.
Total number of groups: 2580
Lower bound on Bayes Error Rate: 0.2528047053697844
Error contributed by 0.7819409650364885 of the whole dataset.


There we have it! Our original dataset of approx. 9000 observations actually only contains 2580 distinct observations. 

Of these, 2002 are singleton observations, from which our computations don't reveal any intrinsic error.

The remaining 578 groups of observations account for a 25.2% error rate! 

Thus, our binary classifiers incurred only 10% more error than the theoretical least possible error (which itself must be a gross under-estimate given that 2002 observations are not contributing).

Thus, relative to this minimum forced error, it is reasonable to conclude that our models performed reasonably well, and the bulk of the error is caused by the insufficient ability of the features (courses) to separate the classes!

In [30]:
grouped_df.head(15)

,165,166,265,143,140,104,150,207,201,317,...,495,314,500,304,COUNT(X),Pr(X),Pr(Y|X),ERROR(Y|X),ERROR(Y|X) * Pr(X),CUMULATIVE_ERROR
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,324,0.035290,0.472222,0.472222,0.016665,0.016665
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,344,0.037469,0.412791,0.412791,0.015467,0.032132
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,214,0.023309,0.602804,0.397196,0.009258,0.041390
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,189,0.020586,0.412698,0.412698,0.008496,0.049886
4,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,177,0.019279,0.604520,0.395480,0.007624,0.057510
5,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,146,0.015902,0.390411,0.390411,0.006208,0.063719
6,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,159,0.017318,0.333333,0.333333,0.005773,0.069491
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,146,0.015902,0.643836,0.356164,0.005664,0.075155
8,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,136,0.014813,0.683824,0.316176,0.004684,0.079839
9,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,95,0.010347,0.442105,0.442105,0.004575,0.084413
